<a href="https://colab.research.google.com/github/lucaslapazini95/projetos_pyspark_soulcode/blob/main/spotify_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from google.cloud import storage
import os
warnings.filterwarnings("ignore")

#CONFIGURAR A VARIÁVEL DE AMBIENTE (SESSÃO)
spark = (SparkSession.builder
                     .master('local')
                     .appName('ad1_pratica')
                     .config('spark.ui.port', '4050')
                     .getOrCreate()
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=735742134ca25771253e908136e1ab2fc6ec55a04826a5d27dc29a631d089ca9
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
df = (spark.read
           .format('csv')
           .option('delimiter', ',')
           .option('header', 'true')
           .option('inferschema', 'false')
           .option('encoding', 'utf-8')
           .option('escape','"')
           .load('/content/drive/MyDrive/Fontes de dados/Tratados/spotify.csv')
)

In [ ]:
#amostra dos dados
df.show()

+---+--------------------+---------------+----------------+--------------------+--------------------+------------------+--------------------+---+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|_c0|           song_name|song_popularity|song_duration_ms|        acousticness|        danceability|            energy|    instrumentalness|key|           liveness|           loudness|audio_mode|         speechiness|  tempo|time_signature|     audio_valence|
+---+--------------------+---------------+----------------+--------------------+--------------------+------------------+--------------------+---+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|  0|Boulevard of Brok...|             73|          262333|0.005520000000000...|          0.496mol/L|             0.682|            2.94e-05|  8|             0.0589|             -4.095|         1|              0.0294| 16

In [ ]:
#VISUALIZAR AS COLUNAS
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- song_name: string (nullable = true)
 |-- song_popularity: string (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)



In [ ]:
len(df.columns)

16

In [ ]:
#SHAPE
print(f'({df.count()}, {len(df.columns)})')

(18835, 16)


In [ ]:
#Verificar nas colunas se possuem valores nulos
df.toPandas().isna().any()

_c0                 False
song_name           False
song_popularity     False
song_duration_ms    False
acousticness        False
danceability        False
energy              False
instrumentalness    False
key                 False
liveness            False
loudness            False
audio_mode          False
speechiness         False
tempo               False
time_signature      False
audio_valence        True
dtype: bool

In [ ]:
#Verificar nas colunas a quantidade de valores nulos
df.toPandas().isna().sum()

_c0                 0
song_name           0
song_popularity     0
song_duration_ms    0
acousticness        0
danceability        0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
audio_mode          0
speechiness         0
tempo               0
time_signature      0
audio_valence       1
dtype: int64

In [ ]:
#VERIFICAR SE NAS COLUNAS POSSUEM VALORES NULOS COM O Pyspark - Valores nulos NULL
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|_c0|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+---+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|  0|        0|              0|               0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            1|
+---+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+



In [ ]:
#DROPAMOS A COLUNA DE ID, POIS ELA SERVE APENAS PARA IDENTIFICAR UM REGISTRO E NÃO SERÁ UTILIZADA NA NOSSA ANÁLISE
df = df.drop(F.col('_c0'))

In [ ]:
df.show()

+--------------------+---------------+----------------+--------------------+--------------------+------------------+--------------------+---+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|           song_name|song_popularity|song_duration_ms|        acousticness|        danceability|            energy|    instrumentalness|key|           liveness|           loudness|audio_mode|         speechiness|  tempo|time_signature|     audio_valence|
+--------------------+---------------+----------------+--------------------+--------------------+------------------+--------------------+---+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|Boulevard of Brok...|             73|          262333|0.005520000000000...|          0.496mol/L|             0.682|            2.94e-05|  8|             0.0589|             -4.095|         1|              0.0294| 167.06|           

In [ ]:
df.printSchema()

root
 |-- song_name: string (nullable = true)
 |-- song_popularity: string (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)



In [ ]:
#Backup

df_backup = df

In [ ]:
#sobrescrever com backup

df = df_backup

In [ ]:
df.count()

18835

In [ ]:
df.drop_duplicates().count()

14932

In [ ]:
#regex para retirar unidades de medida

df = df.withColumn("acousticness", F.regexp_replace("acousticness", "kg", ""))
df = df.withColumn("danceability", F.regexp_replace("danceability", "mol/L", ""))

In [ ]:
# lista com o nome das colunas a serem modificadas pelo regex

colunas = (['song_popularity', 'song_duration_ms', 'acousticness',
            'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
            'loudness', 'audio_mode', 'speechiness', 'tempo',
            'time_signature', 'audio_valence'])

In [ ]:
#criação de lista conforme o que professor passou
#muito mais fácil

colunas = df.drop(F.col('song_name')).columns

In [ ]:
#função que remove strings

def remove_strings_inplace(df, colunas):
    # itera sobre as colunas especificadas e aplica a transformação em cada uma
    for column in colunas:
        df = df.withColumn(column, F.regexp_replace(F.col(column), "[^0-9.eE-]", ""))
    return df

In [ ]:
#chama a função passando de parâmetro o df e a lista criada

df = remove_strings_inplace(df, colunas)

In [ ]:
df.show()

+--------------------+---------------+----------------+--------------------+------------------+------------------+--------------------+---+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|           song_name|song_popularity|song_duration_ms|        acousticness|      danceability|            energy|    instrumentalness|key|           liveness|           loudness|audio_mode|         speechiness|  tempo|time_signature|     audio_valence|
+--------------------+---------------+----------------+--------------------+------------------+------------------+--------------------+---+-------------------+-------------------+----------+--------------------+-------+--------------+------------------+
|Boulevard of Brok...|             73|          262333|0.005520000000000001|             0.496|             0.682|            2.94e-05|  8|             0.0589|             -4.095|         1|              0.0294| 167.06|             4|    

In [ ]:
df.count()

18835

In [ ]:
df.drop_duplicates().count()

14932

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.printSchema()

root
 |-- song_name: string (nullable = true)
 |-- song_popularity: string (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)



In [ ]:
df = df.withColumn('acousticness', F.round(F.col('acousticness'), 4))

In [ ]:
df = df.withColumn('danceability', F.round(F.col('danceability'), 4))

In [ ]:
df = df.withColumn('energy', F.round(F.col('energy'), 4))

In [ ]:
df = df.withColumn('instrumentalness', F.round(F.col('instrumentalness'), 4))

In [ ]:
df = df.withColumn('liveness', F.round(F.col('liveness'), 4))

In [ ]:
df = df.withColumn('loudness', F.round(F.col('loudness'), 4))

In [ ]:
df = df.withColumn('speechiness', F.round(F.col('speechiness'), 4))

In [ ]:
df = df.withColumn('audio_valence', F.round(F.col('audio_valence'), 4))

In [ ]:
df.show()

+--------------------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-------+--------------+-------------+
|           song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|  tempo|time_signature|audio_valence|
+--------------------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-------+--------------+-------------+
|Boulevard of Brok...|             73|          262333|      0.0055|       0.496| 0.682|             0.0|  8|  0.0589|  -4.095|         1|     0.0294| 167.06|             4|        0.474|
|          In The End|             66|          216933|      0.0103|       0.542| 0.853|             0.0|  3|   0.108|  -6.407|         0|     0.0498|105.256|             4|         0.37|
|   Seven Nation Army|             76|          231733|     

In [ ]:
df.printSchema()

In [ ]:
# lista com o nome das colunas a serem modificadas pelo regex
colunas = (['song_popularity', 'song_duration_ms', 'acousticness',
            'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
            'loudness', 'audio_mode', 'speechiness', 'tempo',
            'time_signature', 'audio_valence'])

In [ ]:
# aplicar o regex em cada coluna
for coluna in colunas:
    df = df.withColumn(coluna, F.regexp_replace(coluna, "[^\\d.]", "").cast('double'))

In [ ]:
#VERIFICAR SE NAS COLUNAS POSSUEM VALORES NULOS COM O Pyspark - Valores nulos NULL
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
df.show()

+--------------------+---------------+----------------+------------+------------+------+----------------+----+--------+--------+----------+-----------+-------+--------------+-------------+
|           song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness| key|liveness|loudness|audio_mode|speechiness|  tempo|time_signature|audio_valence|
+--------------------+---------------+----------------+------------+------------+------+----------------+----+--------+--------+----------+-----------+-------+--------------+-------------+
|Boulevard of Brok...|           73.0|        262333.0|      0.0055|       0.496| 0.682|             0.0| 8.0|  0.0589|   4.095|       1.0|     0.0294| 167.06|           4.0|        0.474|
|          In The End|           66.0|        216933.0|      0.0103|       0.542| 0.853|             0.0| 3.0|   0.108|   6.407|       0.0|     0.0498|105.256|           4.0|         0.37|
|   Seven Nation Army|           76.0|        231733.0|

In [ ]:
df.select(F.col('audio_mode')).summary().show()

In [ ]:
df.select(F.col('audio_mode')).distinct().show()